<a href="https://colab.research.google.com/github/liamti5/Essentials-in-Text-and-Speech-Processing/blob/master/machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install sacremoses

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import sentencepiece

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    print("using GPU")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")
classifier = pipeline('translation', model=model, tokenizer=tokenizer, device=device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/Colab/Uni/HS23/EITASP/reviews_en.csv', 'r') as f:
    df = pd.read_csv(f)

df.head()

In [ ]:
# Process translations with batch processing and error handling
translations = []

len_df = len(df)
for index, row in df.iterrows():
    print(len_df)
    review = row["review"]
    print(review)
    if len(review) <= 512:
      try:
          translation = classifier(review, max_length=512)[0]["translation_text"]
          print(translation)
          translations.append(translation)
      except Exception as e:
          print(e)
          translations.append("")
      finally:
          len_df -= 1
    else:
      translations.append("")

In [ ]:
print(translations)

In [ ]:
df.insert(loc = 2,
          column = 'translations',
          value = translations)

In [ ]:
df.to_csv('/content/drive/My Drive/Colab/Uni/HS23/EITASP/reviews_en_translated.csv', index=False)